# **Rotten Tomatoes Analysis**

Importing relevant libraries

In [1]:
import numpy as np
import pandas as pd
import re

from matplotlib import pyplot as plt
%matplotlib inline



fresh_tomato = 0.6 # if the review rating is at least 60% positive, it is a fresh tomato.
rotten_tomato = 0.5 


### **Working Files**


rt_movie_info = pd.read_csv('data/zippedData/rt.movie_info.tsv.gz',encoding='unicode_escape', sep='\t')
rt_reviews = pd.read_csv('data/zippedData/rt.reviews.tsv.gz',encoding='unicode_escape', sep='\t')

In [2]:
rt_movie_info = pd.read_csv('data/zippedData/rt.movie_info.tsv.gz',encoding='unicode_escape', sep='\t')
rt_reviews = pd.read_csv('data/zippedData/rt.reviews.tsv.gz',encoding='unicode_escape', sep='\t')

## **High Level Analysis**

### Comment
We have 'id' and 'rating' that match up, but it is too early to tell if they have anything in common.

In [3]:
print(rt_movie_info.columns)
print()
print(rt_reviews.columns)

Index(['id', 'synopsis', 'rating', 'genre', 'director', 'writer',
       'theater_date', 'dvd_date', 'currency', 'box_office', 'runtime',
       'studio'],
      dtype='object')

Index(['id', 'review', 'rating', 'fresh', 'critic', 'top_critic', 'publisher',
       'date'],
      dtype='object')


### Comment:
There is a good chunk of missing data for 'currency', 'box office', and 'studio'. We also have other columns that have a bit of missing values. Depending on their value, I may or may not need to clean them.

In [4]:
print(f'*** rt_reviews DataFrame ***')
rt_reviews.info()

*** rt_reviews DataFrame ***
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


### Comment:
We do have some more missing values, but the biggest issues are the data types. I may need to convert them to int or float. Most likely the 'rating' column.

In [5]:
print(f'*** rt_movie_info DataFrame ***')
rt_movie_info.info()

*** rt_movie_info DataFrame ***
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


### Comment:
Rating in rt_movie_info looks to pertain to the actual movie rating. I do see we can catagorize the genre to make a graph based on that.
I don't see any value for the following columns: 'synopsis', 'currency', 'box office' (missing data, may need to talk with 
Hatice about this one), and studio (may change my mind later).

In [6]:
rt_movie_info

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,NaN,NaN,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",106 minutes,New Line Cinema
1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",NaN,NaN,88 minutes,Paramount Vantage
1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,NaN,"Jan 1, 1962","May 11, 2004",NaN,NaN,111 minutes,NaN
1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",NaN,NaN,101 minutes,NaN


In [7]:
rt_reviews

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"
...,...,...,...,...,...,...,...,...
54427,2000,The real charm of this trifle is the deadpan c...,NaN,fresh,Laura Sinagra,1,Village Voice,"September 24, 2002"
54428,2000,NaN,1/5,rotten,Michael Szymanski,0,Zap2it.com,"September 21, 2005"
54429,2000,NaN,2/5,rotten,Emanuel Levy,0,EmanuelLevy.Com,"July 17, 2005"
54430,2000,NaN,2.5/5,rotten,Christopher Null,0,Filmcritic.com,"September 7, 2003"


### Analysis Conclusion:
We do have some relatable columns! The 'id' column will come in handy when handling these datasets. The 'rating' column in rt_reviews looks to be a mess, and will take considerable time to clean. The next most useful column would be 'fresh'! Rotten Tomatoes has a rule where if the review is at LEAST 60% in rating, then it is a fresh tomato!. I'm not sure if I can get any valuable data/info from the rest of the columns. Bummer.

## **Cleaning rt_movie_info**
#### Working Variables
- rt_movie_info
- wip_genre
- dropem
- rt_movie_copy

In [8]:
rt_movie_info.columns

Index(['id', 'synopsis', 'rating', 'genre', 'director', 'writer',
       'theater_date', 'dvd_date', 'currency', 'box_office', 'runtime',
       'studio'],
      dtype='object')

### Decision:
Going over the dataset, I am going to remove the following columns: The focus is provide exploring what types of movies are CURRENTLY doing the BEST at the box office.

- 'synopsis' - I don't have a movie title and cannot go through each value to "guess" what movie it is.
- 'director' - Same as 'synopsis'. Directors can have mulitple movies. I can't make a logical guess on this.
- 'writer' - Same as above.
- 'dvd_date' - I'm looking for opening figures.
- 'currency' - Would be relevant if I had more data on 'box office'. It would be possible to see what genre makes the most money.
- 'box_office' - NEED MORE DATA!! I can't use it.
- 'studio' - I'm missing to much data. I can't make a logical guess here.

I'll aslo need to check for duplicates and get rid/alter null values

In [9]:
# creating a copy of the dataframe just in case ;)

rt_movie_copy = rt_movie_info.copy()

# Checking for duplicates
print(rt_movie_copy.duplicated().value_counts())
print(len(rt_movie_copy))

False    1560
dtype: int64
1560


In [10]:
dropem = ['synopsis', 'director', 'writer', 'dvd_date', 'currency', 
            'box_office', 'studio', 'runtime']
rt_movie_copy = rt_movie_info.drop(dropem, 1)

rt_movie_copy

,id,rating,genre,theater_date
0,1,R,Action and Adventure|Classics|Drama,"Oct 9, 1971"
1,3,R,Drama|Science Fiction and Fantasy,"Aug 17, 2012"
2,5,R,Drama|Musical and Performing Arts,"Sep 13, 1996"
3,6,R,Drama|Mystery and Suspense,"Dec 9, 1994"
4,7,NR,Drama|Romance,NaN
...,...,...,...,...
1555,1996,R,Action and Adventure|Horror|Mystery and Suspense,"Aug 18, 2006"
1556,1997,PG,Comedy|Science Fiction and Fantasy,"Jul 23, 1993"
1557,1998,G,Classics|Comedy|Drama|Musical and Performing Arts,"Jan 1, 1962"
1558,1999,PG,Comedy|Drama|Kids and Family|Sports and Fitness,"Apr 1, 1993"


In [11]:
rt_movie_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   rating        1557 non-null   object
 2   genre         1552 non-null   object
 3   theater_date  1201 non-null   object
dtypes: int64(1), object(3)
memory usage: 48.9+ KB


In [12]:
rt_movie_copy[rt_movie_copy['genre'].isna()]

,id,rating,genre,theater_date
10,17,NaN,NaN,NaN
131,167,NaN,NaN,NaN
222,289,NR,NaN,NaN
250,327,NR,NaN,NaN
658,843,NR,NaN,NaN
1082,1393,R,NaN,NaN
1342,1736,NR,NaN,NaN
1543,1982,NaN,NaN,NaN


In [13]:
# These rows don't provide any value. Let's drop them.
dropem = rt_movie_copy[rt_movie_copy['genre'].isna()].index
dropem = list(dropem)
rt_movie_copy = rt_movie_copy.drop(dropem)

# I'll reset the index since we are missing rows.
rt_movie_copy = rt_movie_copy.reset_index(drop=True)

rt_movie_copy['genre'].isna().value_counts()

False    1552
Name: genre, dtype: int64

In [14]:
rt_movie_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1552 entries, 0 to 1551
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1552 non-null   int64 
 1   rating        1552 non-null   object
 2   genre         1552 non-null   object
 3   theater_date  1201 non-null   object
dtypes: int64(1), object(3)
memory usage: 48.6+ KB


In [15]:
# I want to most recent data, so i will need the 'theater_date' column.
# I will drop null values since I can't decern what it could be from this
# dataset.
dropem = rt_movie_copy[rt_movie_copy['theater_date'].isna()]
dropem = list(dropem.index)
rt_movie_copy = rt_movie_copy.drop(dropem)

rt_movie_copy = rt_movie_copy.reset_index(drop=True)


# Since we are looking for CURRENT film type recommendations, I'll set
# the data from 2000 to current. Also change it to datetime type.
rt_movie_copy['theater_date'] = pd.to_datetime(rt_movie_copy['theater_date'])

dropem = rt_movie_copy[rt_movie_copy['theater_date'] < '2000']
dropem = list(dropem.index)
rt_movie_copy = rt_movie_copy.drop(dropem)

# I'll need to reset the index again. Just in case.
rt_movie_copy = rt_movie_copy.reset_index(drop=True)


In [16]:
print(rt_movie_copy['theater_date'].max())
print(rt_movie_copy['theater_date'].min())

2018-10-19 00:00:00
2000-01-01 00:00:00


### Comment
I'll need to clean up the 'genre' column since it's all one string. To make it easier to play with, I'll add each genre in the row into a list using .split()

In [19]:
wip_genre = rt_movie_copy['genre'].copy()

for i in range(len(wip_genre)):
    wip_genre[i] = wip_genre[i].split('|')

rt_movie_copy['genre'] = wip_genre
rt_movie_copy

,id,rating,genre,theater_date
0,3,R,"[Drama, Science Fiction and Fantasy]",2012-08-17
1,8,PG,"[Drama, Kids and Family]",2000-03-03
2,10,PG-13,[Comedy],2002-01-11
3,13,R,[Drama],2006-04-27
4,14,R,[Drama],2010-06-30
...,...,...,...,...
514,1981,NR,"[Comedy, Drama]",2012-10-05
515,1985,R,"[Horror, Mystery and Suspense]",2007-06-01
516,1986,PG,"[Art House and International, Comedy, Drama]",2002-08-30
517,1996,R,"[Action and Adventure, Horror, Mystery and Sus...",2006-08-18


In [23]:
rt_movie_copy['genre'].apply(type).value_counts()

<class 'list'>    519
Name: genre, dtype: int64

In [22]:
rt_movie_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519 entries, 0 to 518
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            519 non-null    int64         
 1   rating        519 non-null    object        
 2   genre         519 non-null    object        
 3   theater_date  519 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 16.3+ KB


### Clean Up Conclusion:
I have pretty much cleaned up to fit my needs. Time to start working on rt_review!


## **Cleaning rt_movie_info**
#### Working Variables
- rt_reviews

duplicates in rt_reviews